In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

batch_size = 128
learning_rate = 0.001
num_epochs = 20
latent_dim = 128

Using device: cpu


In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

100%|██████████| 26.4M/26.4M [00:00<00:00, 114MB/s] 
100%|██████████| 29.5k/29.5k [00:00<00:00, 3.81MB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 62.0MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 10.2MB/s]


In [5]:
def add_noise(inputs, noise_factor=0.3):
    noise = torch.randn_like(inputs)
    return inputs + noise_factor * noise

In [6]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(128 * 7 * 7, latent_dim)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128 * 7 * 7),
            nn.Unflatten(1, (128, 7, 7)),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed, latent

In [7]:
model = Autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for data in train_loader:
        img, _ = data
        img = img.to(device)
        
        noisy_img = add_noise(img).to(device)
        
        output, _ = model(noisy_img)
        loss = criterion(output, img)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}')

Epoch [1/20], Loss: 0.0807
Epoch [2/20], Loss: 0.0378
Epoch [3/20], Loss: 0.0309
Epoch [4/20], Loss: 0.0275
Epoch [5/20], Loss: 0.0254
Epoch [6/20], Loss: 0.0239
Epoch [7/20], Loss: 0.0228
Epoch [8/20], Loss: 0.0220
Epoch [9/20], Loss: 0.0213
Epoch [10/20], Loss: 0.0209
Epoch [11/20], Loss: 0.0204
Epoch [12/20], Loss: 0.0201
Epoch [13/20], Loss: 0.0199
Epoch [14/20], Loss: 0.0196
Epoch [15/20], Loss: 0.0194
Epoch [16/20], Loss: 0.0192


KeyboardInterrupt: 

In [ ]:
model.eval()
with torch.no_grad():
    for data in test_loader:
        img, _ = data
        img = img.to(device)
        noisy_img = add_noise(img).to(device)
        output, _ = model(noisy_img)
        break

n = 10
plt.figure(figsize=(20, 6))
for i in range(n):
    # Original
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(img[i].cpu().squeeze().numpy(), cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # Noisy
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(noisy_img[i].cpu().squeeze().numpy(), cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # Rekonstruksi
    ax = plt.subplot(3, n, i + 1 + 2*n)
    plt.imshow(output[i].cpu().squeeze().numpy(), cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
with torch.no_grad():
    latents = []
    labels = []
    for data in test_loader:
        img, label = data
        img = img.to(device)
        _, latent = model(img)
        latents.append(latent.cpu().numpy())
        labels.append(label.numpy())
    latents = np.concatenate(latents)
    labels = np.concatenate(labels)

tsne = TSNE(n_components=2, perplexity=30, random_state=42)
latent_2d = tsne.fit_transform(latents)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(latent_2d[:, 0], latent_2d[:, 1], c=labels, cmap='tab10', alpha=0.7)
plt.colorbar(scatter, ticks=range(10))
plt.title('t-SNE Visualization of Latent Space')
plt.show()

In [ ]:
test_loss = 0
with torch.no_grad():
    for data in test_loader:
        img, _ = data
        img = img.to(device)
        output, _ = model(img)
        test_loss += criterion(output, img).item()
print(f'Test Loss: {test_loss / len(test_loader):.4f}')